In [1]:
%%capture
%pip install nltk transformers
%pip install sentencepiece

In [2]:
import nltk
from nltk.corpus import brown
from transformers import TFAlbertModel, AlbertTokenizer
import numpy as np
import csv
from nltk.tokenize import word_tokenize
import os
from tqdm import tqdm

import tensorflow as tf

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
nltk.download('brown')
nltk.download('punkt')

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\nawee\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nawee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
model_name = "albert-base-v2"
model = TFAlbertModel.from_pretrained(model_name)
tokenizer = AlbertTokenizer.from_pretrained(model_name)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFAlbertModel: ['predictions.decoder.bias', 'predictions.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.LayerNorm.bias', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing TFAlbertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFAlbertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.


In [6]:
def embed_words(words):
    tokens = tokenizer.batch_encode_plus(words, padding='max_length', truncation=True, return_tensors='tf', max_length=64)
    outputs = model(tokens['input_ids'])
    embeddings = outputs.last_hidden_state
    averaged_embeddings = tf.reduce_mean(embeddings, axis=1)
    return averaged_embeddings

In [7]:
dataset_path = r"Coursework Brief/Reuters-21578/Reuters-21578/reuters/reuters/reuters/training"
dataset_dirs = os.listdir(dataset_path)

In [8]:
data = []
for i in tqdm(dataset_dirs):
    with open(f"{dataset_path}/{i}", 'r') as f:
        content = f.read()
        data.append(content)

100%|██████████| 7769/7769 [00:00<00:00, 17155.45it/s]


In [9]:
words = []
for text in data:
  temp_words = word_tokenize(text)
  words.append(temp_words)

In [10]:
from itertools import chain

words_flattened = list(chain(*words))


In [11]:
len(words_flattened)

1135633

In [12]:
words_flattened = [str(element) for element in words_flattened]

In [13]:
embeddings = embed_words(words_flattened)

: 

In [ ]:
# Save embeddings and metadata in TSV format
tsv_file = 'word_embeddings.tsv'
metadata_file = 'metadata.tsv'

# Save embeddings
np.savetxt(tsv_file, embeddings, delimiter='\t')

# Save metadata
with open(metadata_file, 'w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file, delimiter='\t')
    writer.writerows([[word] for word in words_flattened])

In [ ]:
len(embeddings)

5000